# Season 3 Episode 18

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style("darkgrid")
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (15, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

import warnings
warnings.simplefilter(action='ignore') 

import opendatasets as od

# Importing Data

In [2]:
od.download('https://www.kaggle.com/competitions/playground-series-s3e18/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading playground-series-s3e18.zip to .\playground-series-s3e18


100%|██████████| 2.41M/2.41M [00:00<00:00, 3.30MB/s]


Extracting archive .\playground-series-s3e18/playground-series-s3e18.zip to .\playground-series-s3e18


In [2]:
od.download('https://www.kaggle.com/datasets/gopalns/ec-mixed-class')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading ec-mixed-class.zip to .\ec-mixed-class


100%|██████████| 398k/398k [00:00<00:00, 1.05MB/s]

In [2]:
path = './playground-series-s3e18/' 

train = pd.read_csv(path+'train.csv').drop(columns='id')
test = pd.read_csv(path+'test.csv').drop(columns='id')
submission = pd.read_csv(path+'sample_submission.csv')  

# EDA and Feature Engineering

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14838 entries, 0 to 14837
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BertzCT            14838 non-null  float64
 1   Chi1               14838 non-null  float64
 2   Chi1n              14838 non-null  float64
 3   Chi1v              14838 non-null  float64
 4   Chi2n              14838 non-null  float64
 5   Chi2v              14838 non-null  float64
 6   Chi3v              14838 non-null  float64
 7   Chi4n              14838 non-null  float64
 8   EState_VSA1        14838 non-null  float64
 9   EState_VSA2        14838 non-null  float64
 10  ExactMolWt         14838 non-null  float64
 11  FpDensityMorgan1   14838 non-null  float64
 12  FpDensityMorgan2   14838 non-null  float64
 13  FpDensityMorgan3   14838 non-null  float64
 14  HallKierAlpha      14838 non-null  float64
 15  HeavyAtomMolWt     14838 non-null  float64
 16  Kappa3             148

In [6]:
train.isna().sum()

BertzCT              0
Chi1                 0
Chi1n                0
Chi1v                0
Chi2n                0
Chi2v                0
Chi3v                0
Chi4n                0
EState_VSA1          0
EState_VSA2          0
ExactMolWt           0
FpDensityMorgan1     0
FpDensityMorgan2     0
FpDensityMorgan3     0
HallKierAlpha        0
HeavyAtomMolWt       0
Kappa3               0
MaxAbsEStateIndex    0
MinEStateIndex       0
NumHeteroatoms       0
PEOE_VSA10           0
PEOE_VSA14           0
PEOE_VSA6            0
PEOE_VSA7            0
PEOE_VSA8            0
SMR_VSA10            0
SMR_VSA5             0
SlogP_VSA3           0
VSA_EState9          0
fr_COO               0
fr_COO2              0
EC1                  0
EC2                  0
EC3                  0
EC4                  0
EC5                  0
EC6                  0
dtype: int64

## Splitting into X and y

In [33]:
X = train.drop(columns=['EC1', 'EC2'])
y = train[['EC1', 'EC2']] 

Original values.

In [34]:
X.describe().loc[['min', 'max']]

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC3,EC4,EC5,EC6
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-5.430556,0.0,0.0,0.0,0.0,0.0,0.0
max,4069.95978,69.551167,50.174588,53.431954,32.195368,34.579313,22.880836,16.07281,363.705954,99.936429,...,80.742293,492.729739,115.406157,384.450519,8.0,8.0,1.0,1.0,1.0,1.0


In [35]:
cols_to_drop = []

for i in X.columns: 
    if i not in test.columns:
        cols_to_drop.append(i) 

cols_to_drop

['EC3', 'EC4', 'EC5', 'EC6']

Even these columns must be dropped. 

In [36]:
X.drop(columns=cols_to_drop, inplace=True)

In [37]:
X.describe().loc[['min', 'max']]

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.430556,0.0,0.0
max,4069.95978,69.551167,50.174588,53.431954,32.195368,34.579313,22.880836,16.07281,363.705954,99.936429,...,482.434223,375.425148,211.501279,100.348416,80.742293,492.729739,115.406157,384.450519,8.0,8.0


In [38]:
test.describe().loc[['min', 'max']]

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.061146,0.0,0.0
max,3919.541449,53.876516,43.283925,53.990574,36.368883,44.726398,26.736931,15.620667,363.705954,99.936429,...,482.434223,258.844527,271.424271,96.917997,79.920667,492.729739,115.406157,384.450519,8.0,8.0


## Scaling 

In [41]:
from sklearn.preprocessing import RobustScaler  

cols_to_scale = X.select_dtypes(np.number).columns.to_list()

scaler = RobustScaler().fit(X[cols_to_scale])  

X[cols_to_scale] = scaler.transform(X[cols_to_scale]) 
test[cols_to_scale] = scaler.transform(test[cols_to_scale])  

In [42]:
X.describe().loc[['min', 'max']]

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
min,-0.577874,-0.999312,-0.873006,-0.785138,-0.773735,-0.708822,-0.583189,-0.528600,-0.446025,-0.500000,...,-0.381538,0.000000,0.000000,0.000000,-1.000000,-0.552935,-0.947712,-1.805138,0.0,0.0
max,7.504676,9.717788,9.935276,8.764780,7.612496,6.849693,6.264682,7.387541,8.901999,7.282215,...,30.454005,30.943161,15.273636,14.493389,5.870193,13.018291,10.458169,13.138188,8.0,8.0


In [43]:
test.describe().loc[['min', 'max']]

,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
min,-0.577874,-0.999312,-0.873006,-0.785138,-0.773735,-0.708822,-0.583189,-0.528600,-0.446025,-0.500000,...,-0.381538,0.000000,0.000000,0.000000,-1.000000,-0.552935,-0.947712,-1.790979,0.0,0.0
max,7.205959,7.302490,8.450935,8.864622,8.699611,9.067693,7.418750,7.164853,8.901999,7.282215,...,30.454005,21.334394,19.600995,13.997931,5.800283,13.018291,10.458169,13.138188,8.0,8.0


Storing them.

In [45]:
X.to_csv('preprocessed_training_data.csv', index=None) 
y.to_csv('target_training.csv', index=None) 
test.to_csv('preprocessed_test_data.csv', index=None)

## Re-Importing Data

In [3]:
X = pd.read_csv('./preprocessed_training_data.csv')
test = pd.read_csv('./preprocessed_test_data.csv') 
y = pd.read_csv('./target_training.csv')

In [4]:
y1 = y.EC1 
y2 = y.EC2

# Machine Learning Model   

Since this is a multi-output classification problem, the MultiOutputClassifier from scikit-learn will be used.

In [4]:
from sklearn.multioutput import MultiOutputClassifier 
from xgboost import XGBClassifier 
from sklearn.model_selection import RandomizedSearchCV 

With multioutput

In [13]:
mlt_clf = RandomizedSearchCV(MultiOutputClassifier(XGBClassifier(n_jobs=-1, random_state=42)), {
    'estimator__max_depth': [1,2,3,4,5,6],
    'estimator__learning_rate': [.001, .003, .005, .007, .01, .015],
    'estimator__n_estimators': [400, 500, 600, 700, 800, 900],
    'estimator__min_child_weight': [4,5,6,7,8,9,10],
    'estimator__subsample': [.7, .75, .8, .85, .9, 1],
    'estimator__colsample_bytree': [.4, .5, .6, .7, .8, .9],
    'estimator__gamma': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'estimator__reg_lambda':[0, 0.5, 1, 1.5, 2, 3, 4, 4.5]
}, cv=5, return_train_score=False, scoring='roc_auc', n_iter=5)

mlt_clf.fit(X,y)

pd.DataFrame(mlt_clf.cv_results_).sort_values('rank_test_score').head(3)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__subsample,param_estimator__reg_lambda,param_estimator__n_estimators,param_estimator__min_child_weight,param_estimator__max_depth,param_estimator__learning_rate,...,param_estimator__colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.664337,0.842301,0.031961,0.004700,0.9,3,400,10,6,0.003,...,0.6,"{'estimator__subsample': 0.9, 'estimator__reg_...",0.652052,0.647240,0.656295,0.637475,0.631827,0.644978,0.009083,1
4,6.338561,0.262179,0.025542,0.000847,0.8,1,800,4,2,0.005,...,0.7,"{'estimator__subsample': 0.8, 'estimator__reg_...",0.654380,0.642911,0.654359,0.633843,0.630702,0.643239,0.009933,2
2,6.604616,0.135331,0.027503,0.004288,0.8,4,900,7,2,0.003,...,0.6,"{'estimator__subsample': 0.8, 'estimator__reg_...",0.653020,0.643122,0.654060,0.632908,0.629407,0.642503,0.010081,3


In [14]:
mlt_clf.best_params_

{'estimator__subsample': 0.9,
 'estimator__reg_lambda': 3,
 'estimator__n_estimators': 400,
 'estimator__min_child_weight': 10,
 'estimator__max_depth': 6,
 'estimator__learning_rate': 0.003,
 'estimator__gamma': 0.4,
 'estimator__colsample_bytree': 0.6}

Without multioutput

In [ ]:
clf_1 = RandomizedSearchCV(XGBClassifier(n_jobs=0, random_state=42), {
    'objective': ['reg:absoluteerror', 'reg:pseudohubererror', 'reg:squaredlogerror'],
    'max_depth': [3,4,5,6,7,8,9],
    'learning_rate': [.001, .005, .01, .05, .1, .2, .3],
    'n_estimators': [500, 700, 900, 1000],
    'min_child_weight': [5,6,7,8,9,10,11,12,13,14,15],
    'subsample': [.4, .5, .6, .7, .8, .9, 1],
    'colsample_bytree': [.4, .5, .6, .7, .8, .9],
    'gamma': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'reg_lambda':[0, 0.5, 1, 1.5, 2, 3, 4.5]
}, cv=5, return_train_score=False, scoring='roc_auc', n_iter=5)

clf_1.fit(X,y1)

pd.DataFrame(clf_1.cv_results_).sort_values('rank_test_score').head(3)


In [20]:
clf_1.best_params_

{'subsample': 0.9,
 'reg_lambda': 2,
 'objective': 'reg:squaredlogerror',
 'n_estimators': 1000,
 'min_child_weight': 5,
 'max_depth': 3,
 'learning_rate': 0.01,
 'gamma': 0.3,
 'colsample_bytree': 0.8}

In [11]:
clf_2 = RandomizedSearchCV(XGBClassifier(n_jobs=0, random_state=42, scale_pos_weight=2983/11855), {
    'objective': ['reg:absoluteerror', 'reg:pseudohubererror', 'reg:squaredlogerror'],
    'max_depth': [1,2,3,4,5,6,7,8,9,10],
    'learning_rate': [.001, .005, .01, .05, .1, .2, .3],
    'n_estimators': [300, 500, 1000, 1500],
    'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
    'subsample': [.4, .5, .6, .7, .8, .9, 1],
    'colsample_bytree': [.4, .5, .6, .7, .75, .77, .79 ,.8, .85],
    'gamma': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'reg_lambda':[0, 0.5, 1, 1.5, 2, 3, 4.5]
}, cv=5, return_train_score=False, scoring='roc_auc', n_iter=5)

clf_2.fit(X,y2)

pd.DataFrame(clf_2.cv_results_).sort_values('rank_test_score').head(3)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_reg_lambda,param_n_estimators,param_min_child_weight,param_max_depth,param_learning_rate,...,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,3.904447,0.052295,0.019224,0.003294,0.7,1,1000,4,1,0.01,...,0.75,"{'subsample': 0.7, 'reg_lambda': 1, 'n_estimat...",0.602971,0.575292,0.597067,0.557292,0.573464,0.581217,0.016686,1
1,2.439310,0.132070,0.011885,0.003583,0.4,1,500,2,2,0.001,...,0.8,"{'subsample': 0.4, 'reg_lambda': 1, 'n_estimat...",0.602940,0.573464,0.598993,0.554241,0.571347,0.580197,0.018264,2
2,2.720494,0.019151,0.015482,0.001592,0.6,1,500,2,2,0.3,...,0.77,"{'subsample': 0.6, 'reg_lambda': 1, 'n_estimat...",0.572344,0.544613,0.573115,0.542451,0.547172,0.555939,0.013793,3


In [ ]:
clf_2.best_params_

# Submission 

With multioutput

In [15]:
submission['EC1'] = mlt_clf.predict_proba(test)[0][:,1] 
submission['EC2'] = mlt_clf.predict_proba(test)[1][:,1]
submission 

,id,EC1,EC2
0,14838,0.473760,0.679856
1,14839,0.716790,0.735998
2,14840,0.697255,0.663994
3,14841,0.651463,0.722458
4,14842,0.691271,0.718828
...,...,...,...
9888,24726,0.601436,0.675453
9889,24727,0.674719,0.737159
9890,24728,0.436566,0.746746
9891,24729,0.464293,0.755920


In [16]:
submission.to_csv('forth_attempt.csv', index=None)

Without multioutput

In [19]:
pred1 = clf_1.predict_proba(test)[:, 1]
pred2 = clf_2.predict_proba(test)[:, 1] 

submission['EC1'] = pred1 
submission['EC2'] = pred2 

In [20]:
submission

,id,EC1,EC2
0,14838,0.458690,0.757737
1,14839,0.740749,0.710969
2,14840,0.637864,0.713031
3,14841,0.554321,0.818392
4,14842,0.600162,0.707182
...,...,...,...
9888,24726,0.500874,0.702792
9889,24727,0.716235,0.727466
9890,24728,0.510389,0.804723
9891,24729,0.582583,0.783010


In [21]:
submission.to_csv('second_attempt.csv', index=None)